<a href="https://colab.research.google.com/github/noriko-hiroi/noriko-hiroi/blob/main/8_3_5_deeplearning_gan_sam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 8.3 SAM（Structural Agnostic Model）による因果探索の実装

本ファイルは、8.3節の実装です。

7.5節と同じく、「上司向け：部下とのキャリア面談のポイント研修」の疑似データを作成して、SAMによる因果探索を実施します。

## プログラム実行前の設定など

In [ ]:
# PyTorchのバージョンを下げる
!pip install torch==1.4.0+cu92 torchvision==0.5.0+cu92 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 640.6MB 28kB/s 
     |████████████████████████████████| 3.9MB 55.0MB/s 
ERROR: torchtext 0.9.1 has requirement torch==1.8.1, but you'll have torch 1.4.0+cu92 which is incompatible.
  Found existing installation: torch 1.8.1+cu101
    Uninstalling torch-1.8.1+cu101:
      Successfully uninstalled torch-1.8.1+cu101
  Found existing installation: torchvision 0.9.1+cu101
    Uninstalling torchvision-0.9.1+cu101:
      Successfully uninstalled torchvision-0.9.1+cu101


In [ ]:
import torch 
print(torch.__version__)  # 元は1.5.0+cu101、versionを1.4に下げた

1.4.0+cu92


In [ ]:
# 乱数のシードを設定
import random
import numpy as np

np.random.seed(1234)
random.seed(1234)


In [ ]:
# 使用するパッケージ（ライブラリと関数）を定義
# 標準正規分布の生成用
from numpy.random import *

# グラフの描画用
import matplotlib.pyplot as plt

# その他
import pandas as pd

# シグモイド関数をimport
from scipy.special import expit


## データの作成

In [ ]:
# データ数
num_data = 2000

# 部下育成への熱心さ
x = np.random.uniform(low=-1, high=1, size=num_data)  # -1から1の一様乱数

# 上司が「上司向け：部下とのキャリア面談のポイント研修」に参加したかどうか
e_z = randn(num_data)  # ノイズの生成
z_prob = expit(-5.0*x+5*e_z)
Z = np.array([])

# 上司が「上司向け：部下とのキャリア面談のポイント研修」に参加したかどうか
for i in range(num_data):
    Z_i = np.random.choice(2, size=1, p=[1-z_prob[i], z_prob[i]])[0]
    Z = np.append(Z, Z_i)

# 介入効果の非線形性：部下育成の熱心さxの値に応じて段階的に変化
t = np.zeros(num_data)
for i in range(num_data):
    if x[i] < 0:
        t[i] = 0.5
    elif x[i] >= 0 and x[i] < 0.5:
        t[i] = 0.7
    elif x[i] >= 0.5:
        t[i] = 1.0

e_y = randn(num_data)
Y = 2.0 + t*Z + 0.3*x + 0.1*e_y 


# 本章からの追加データを生成

# Y2：部下当人のチームメンバへの満足度 1から5の5段階
Y2 = np.random.choice([1.0, 2.0, 3.0, 4.0, 5.0],
                      num_data, p=[0.1, 0.2, 0.3, 0.2, 0.2])

# Y3：部下当人の仕事への満足度
e_y3 = randn(num_data)
Y3 = 3*Y + Y2 + e_y3

# Y4：部下当人の仕事のパフォーマンス
e_y4 = randn(num_data)
Y4 = 3*Y3 + 2*e_y4 + 5


## データをまとめた表を作成し、正規化し、可視化する

In [ ]:
df = pd.DataFrame({'x': x,
                   'Z': Z,
                   't': t,
                   'Y': Y,
                   'Y2': Y2,
                   'Y3': Y3,
                   'Y4': Y4,
                   })

del df["t"]  # 変数tは観測できないので削除

df.head()  # 先頭を表示


x    Z         Y   Y2         Y3         Y4
0 -0.616961  1.0  2.286924  2.0   8.732544  30.326507
1  0.244218  1.0  2.864636  3.0  10.743959  37.149014
2 -0.124545  0.0  2.198515  3.0  10.569163  38.481185
3  0.570717  1.0  3.230572  3.0  12.312526  43.709229
4  0.559952  0.0  2.459267  5.0  12.418739  40.833938

## SAMによる推論を実施

In [ ]:
!pip install cdt==0.5.18

     |████████████████████████████████| 921kB 8.1MB/s 
  Created wheel for skrebate: filename=skrebate-0.62-cp37-none-any.whl size=29256 sha256=18c8d5af558e3bd7fa5d534aa546fb91ff3d4ebddbe59025f2fbc4885db41e0c
  Stored in directory: /root/.cache/pip/wheels/6b/0e/2f/540392932d3a01060f04c242d3b9bee80dfd87f63d80b3a9a0
  Created wheel for GPUtil: filename=GPUtil-1.4.0-cp37-none-any.whl size=7411 sha256=5cdc296e7ba045436b1d3f7b0536e1f6dde6bfb562365382c534e6da14479558
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built skrebate GPUtil


### SAMの識別器Dの実装

In [ ]:
# PyTorchから使用するものをimport
import torch
import torch.nn as nn


class SAMDiscriminator(nn.Module):
    """SAMのDiscriminatorのニューラルネットワーク
    """

    def __init__(self, nfeatures, dnh, hlayers):
        super(SAMDiscriminator, self).__init__()

        # ----------------------------------
        # ネットワークの用意
        # ----------------------------------
        self.nfeatures = nfeatures  # 入力変数の数

        layers = []
        layers.append(nn.Linear(nfeatures, dnh))
        layers.append(nn.BatchNorm1d(dnh))
        layers.append(nn.LeakyReLU(.2))

        for i in range(hlayers-1):
            layers.append(nn.Linear(dnh, dnh))
            layers.append(nn.BatchNorm1d(dnh))
            layers.append(nn.LeakyReLU(.2))

        layers.append(nn.Linear(dnh, 1))  # 最終出力

        self.layers = nn.Sequential(*layers)

        # ----------------------------------
        # maskの用意（対角成分のみ1で、他は0の行列）
        # ----------------------------------
        mask = torch.eye(nfeatures, nfeatures)  # 変数の数×変数の数の単位行列
        self.register_buffer("mask", mask.unsqueeze(0))  # 単位行列maskを保存しておく

        # 注意：register_bufferはmodelのパラメータではないが、その後forwardで使う変数を登録するPyTorchのメソッドです
        # self.変数名で、以降も使用可能になります
        # https://pytorch.org/docs/stable/nn.html?highlight=register_buffer#torch.nn.Module.register_buffer

    def forward(self, input, obs_data=None):
        """　順伝搬の計算
        Args:
            input (torch.Size([データ数, 観測変数の種類数])): 観測したデータ、もしくは生成されたデータ
            obs_data (torch.Size([データ数, 観測変数の種類数])):観測したデータ
        Returns:
            torch.Tensor: 観測したデータか、それとも生成されたデータかの判定結果
        """

        if obs_data is not None:
          # 生成データを識別器に入力する場合
            return [self.layers(i) for i in torch.unbind(obs_data.unsqueeze(1) * (1 - self.mask)
                                                         + input.unsqueeze(1) * self.mask, 1)]
            # 対角成分のみ生成したデータ、その他は観測データに
            # データを各変数ごとに、生成したもの、その他観測したもので混ぜて、1変数ずつ生成したものを放り込む
            # torch.unbind(x,1)はxの1次元目でテンソルをタプルに展開する
            # minibatch数が2000、観測データの変数が6種類の場合、
            # [2000,6]→[2000,6,6]→([2000,6], [2000,6], [2000,6], [2000,6], [2000,6], [2000,6])→([2000,1], [2000,1], [2000,1], [2000,1], [2000,1], [2000,1])
            # returnは[torch.Size([2000, 1]),torch.Size([2000, 1]),torch.Size([2000, 1], torch.Size([2000, 1]),torch.Size([2000, 1]),torch.Size([2000, 1])]

            # 注：生成した変数全種類を用いた判定はしない。
            # すなわち、生成した変数1種類と、元の観測データたちをまとめて1つにし、それが観測結果か、生成結果を判定させる

        else:
            # 観測データを識別器に入力する場合

            return self.layers(input)
            # returnは[torch.Size([2000, 1])]


    def reset_parameters(self):
        """識別器Dの重みパラメータの初期化を実施"""
        for layer in self.layers:
            if hasattr(layer, 'reset_parameters'):
                layer.reset_parameters()


### SAMの生成器Gの実装

In [ ]:
from cdt.utils.torch import ChannelBatchNorm1d, MatrixSampler, Linear3D


class SAMGenerator(nn.Module):
    """SAMのGeneratorのニューラルネットワーク
    """

    def __init__(self, data_shape, nh):
        """初期化"""
        super(SAMGenerator, self).__init__()

        # ----------------------------------
        # 対角成分のみ0で、残りは1のmaskとなる変数skeletonを作成
        # ※最後の行は、全部1です
        # ----------------------------------
        nb_vars = data_shape[1]  # 変数の数
        skeleton = 1 - torch.eye(nb_vars + 1, nb_vars)

        self.register_buffer('skeleton', skeleton)

        # 注意：register_bufferはmodelのパラメータではないが、その後forwardで使う変数を登録するPyTorchのメソッドです
        # self.変数名で、以降も使用可能になります
        # https://pytorch.org/docs/stable/nn.html?highlight=register_buffer#torch.nn.Module.register_buffer

        # ----------------------------------
        # ネットワークの用意
        # ----------------------------------
        # 入力層（SAMの形での全結合層）　
        self.input_layer = Linear3D(
            (nb_vars, nb_vars + 1, nh))  # nhは中間層のニューロン数
        # https://github.com/FenTechSolutions/CausalDiscoveryToolbox/blob/32200779ab9b63762be3a24a2147cff09ba2bb72/cdt/utils/torch.py#L289

        # 中間層
        layers = []
        # 2次元を1次元に変換してバッチノーマライゼーションするモジュール
        layers.append(ChannelBatchNorm1d(nb_vars, nh))
        layers.append(nn.Tanh())
        self.layers = nn.Sequential(*layers)

        # ChannelBatchNorm1d
        # https://github.com/FenTechSolutions/CausalDiscoveryToolbox/blob/32200779ab9b63762be3a24a2147cff09ba2bb72/cdt/utils/torch.py#L130

        # 出力層（再度、SAMの形での全結合層）
        self.output_layer = Linear3D((nb_vars, nh, 1))

    def forward(self, data, noise, adj_matrix, drawn_neurons=None):
        """　順伝搬の計算
        Args:
            data (torch.Tensor): 観測データ
            noise (torch.Tensor): データ生成用のノイズ
            adj_matrix (torch.Tensor): 因果関係を示す因果構造マトリクスM
            drawn_neurons (torch.Tensor): Linear3Dの複雑さを制御する複雑さマトリクスZ
        Returns:
            torch.Tensor: 生成されたデータ
        """

        # 入力層
        x = self.input_layer(data, noise, adj_matrix *
                             self.skeleton)  # Linear3D

        # 中間層（バッチノーマライゼーションとTanh）
        x = self.layers(x)

        # 出力層
        output = self.output_layer(
            x, noise=None, adj_matrix=drawn_neurons)  # Linear3D

        return output.squeeze(2)

    def reset_parameters(self):
        """重みパラメータの初期化を実施"""

        self.input_layer.reset_parameters()
        self.output_layer.reset_parameters()

        for layer in self.layers:
            if hasattr(layer, 'reset_parameters'):
                layer.reset_parameters()


Detecting 1 CUDA device(s).


### SAMの誤差関数

In [ ]:
# ネットワークを示す因果構造マトリクスMがDAG（有向非循環グラフ）になるように加える損失

def notears_constr(adj_m, max_pow=None):
    """No Tears constraint for binary adjacency matrixes. 
    Args:
        adj_m (array-like): Adjacency matrix of the graph
        max_pow (int): maximum value to which the infinite sum is to be computed.
           defaults to the shape of the adjacency_matrix
    Returns:
        np.ndarray or torch.Tensor: Scalar value of the loss with the type
            depending on the input.
    参考：https://github.com/FenTechSolutions/CausalDiscoveryToolbox/blob/32200779ab9b63762be3a24a2147cff09ba2bb72/cdt/utils/loss.py#L215
    """
    m_exp = [adj_m]
    if max_pow is None:
        max_pow = adj_m.shape[1]
    while(m_exp[-1].sum() > 0 and len(m_exp) < max_pow):
        m_exp.append(m_exp[-1] @ adj_m/len(m_exp))

    return sum([i.diag().sum() for idx, i in enumerate(m_exp)])
    

### SAMの学習を実施する関数

In [ ]:
from sklearn.preprocessing import scale
from torch import optim
from torch.utils.data import DataLoader
from tqdm import tqdm


def run_SAM(in_data, lr_gen, lr_disc, lambda1, lambda2, hlayers, nh, dnh, train_epochs, test_epochs, device):
    '''SAMの学習を実行する関数'''

    # ---------------------------------------------------
    # 入力データの前処理
    # ---------------------------------------------------
    list_nodes = list(in_data.columns)  # 入力データの列名のリスト
    data = scale(in_data[list_nodes].values)  # 入力データの正規化
    nb_var = len(list_nodes)  # 入力データの数 = d
    data = data.astype('float32')  # 入力データをfloat32型に
    data = torch.from_numpy(data).to(device)  # 入力データをPyTorchのテンソルに
    rows, cols = data.size()  # rowsはデータ数、colsは変数の数

    # ---------------------------------------------------
    # DataLoaderの作成（バッチサイズは全データ）
    # ---------------------------------------------------
    batch_size = rows  # 入力データ全てを使用したミニバッチ学習とする
    data_iterator = DataLoader(data, batch_size=batch_size,
                               shuffle=True, drop_last=True)
    # 注意：引数のdrop_lastはdataをbatch_sizeで取り出していったときに最後に余ったものは使用しない設定

    # ---------------------------------------------------
    # 【Generator】ネットワークの生成とパラメータの初期化
    # cols：入力変数の数、nhは中間ニューロンの数、hlayersは中間層の数
    # neuron_samplerは、Functional gatesの変数zを学習するネットワーク
    # graph_samplerは、Structual gatesの変数aを学習するネットワーク
    # ---------------------------------------------------
    sam = SAMGenerator((batch_size, cols), nh).to(device)  # 生成器G
    graph_sampler = MatrixSampler(nb_var, mask=None, gumbel=False).to(
        device)  # 因果構造マトリクスMを作るネットワーク
    neuron_sampler = MatrixSampler((nh, nb_var), mask=False, gumbel=True).to(
        device)  # 複雑さマトリクスZを作るネットワーク

    # 注意：MatrixSamplerはGumbel-Softmaxを使用し、0か1を出力させるニューラルネットワーク
    # SAMの著者らの実装モジュール、MatrixSamplerを使用
    # https://github.com/FenTechSolutions/CausalDiscoveryToolbox/blob/32200779ab9b63762be3a24a2147cff09ba2bb72/cdt/utils/torch.py#L212

    # 重みパラメータの初期化
    sam.reset_parameters()
    graph_sampler.weights.data.fill_(2)

    # ---------------------------------------------------
    # 【Discriminator】ネットワークの生成とパラメータの初期化
    # cols：入力変数の数、dnhは中間ニューロンの数、hlayersは中間層の数。
    # ---------------------------------------------------
    discriminator = SAMDiscriminator(cols, dnh, hlayers).to(device)
    discriminator.reset_parameters()  # 重みパラメータの初期化

    # ---------------------------------------------------
    # 最適化の設定
    # ---------------------------------------------------
    # 生成器

    g_optimizer = optim.Adam(sam.parameters(), lr=lr_gen)
    graph_optimizer = optim.Adam(graph_sampler.parameters(), lr=lr_gen)
    neuron_optimizer = optim.Adam(neuron_sampler.parameters(), lr=lr_gen)

    # 識別器
    d_optimizer = optim.Adam(discriminator.parameters(), lr=lr_disc)

    # 損失関数
    criterion = nn.BCEWithLogitsLoss()
    # nn.BCEWithLogitsLoss()は、binary cross entropy with Logistic function
    # https://pytorch.org/docs/stable/nn.html#bcewithlogitsloss

    # 損失関数のDAGに関する制約の設定パラメータ
    dagstart = 0.5
    dagpenalization_increase = 0.001*10

    # ---------------------------------------------------
    # forward計算、および損失関数の計算に使用する変数を用意
    # ---------------------------------------------------
    _true = torch.ones(1).to(device)
    _false = torch.zeros(1).to(device)

    noise = torch.randn(batch_size, nb_var).to(device)  # 生成器Gで使用する生成ノイズ
    noise_row = torch.ones(1, nb_var).to(device)

    output = torch.zeros(nb_var, nb_var).to(device)  # 求まった隣接行列
    output_loss = torch.zeros(1, 1).to(device)

    # ---------------------------------------------------
    # forwardの計算で、ネットワークを学習させる
    # ---------------------------------------------------
    pbar = tqdm(range(train_epochs + test_epochs))  # 進捗（progressive bar）の表示

    for epoch in pbar:
        for i_batch, batch in enumerate(data_iterator):

            # 最適化を初期化
            g_optimizer.zero_grad()
            graph_optimizer.zero_grad()
            neuron_optimizer.zero_grad()
            d_optimizer.zero_grad()

            # 因果構造マトリクスM（drawn_graph）と複雑さマトリクスZ（drawn_neurons）をMatrixSamplerから取得
            drawn_graph = graph_sampler()
            drawn_neurons = neuron_sampler()
            # (drawn_graph)のサイズは、torch.Size([nb_var, nb_var])。 出力値は0か1
            # (drawn_neurons)のサイズは、torch.Size([nh, nb_var])。 出力値は0か1

            # ノイズをリセットし、生成器Gで疑似データを生成
            noise.normal_()
            generated_variables = sam(data=batch, noise=noise,
                                      adj_matrix=torch.cat(
                                          [drawn_graph, noise_row], 0),
                                      drawn_neurons=drawn_neurons)

            # 識別器Dで判定
            # 観測変数のリスト[]で、各torch.Size([data数, 1])が求まる
            disc_vars_d = discriminator(generated_variables.detach(), batch)
            # 観測変数のリスト[] で、各torch.Size([data数, 1])が求まる
            disc_vars_g = discriminator(generated_variables, batch)
            true_vars_disc = discriminator(batch)  # torch.Size([data数, 1])が求まる

            # 損失関数の計算（DCGAN）
            disc_loss = sum([criterion(gen, _false.expand_as(gen)) for gen in disc_vars_d]) / nb_var \
                + criterion(true_vars_disc, _true.expand_as(true_vars_disc))

            gen_loss = sum([criterion(gen,
                                      _true.expand_as(gen))
                            for gen in disc_vars_g])

            # 損失の計算（SAM論文のオリジナルのfgan）
            #disc_loss = sum([torch.mean(torch.exp(gen - 1)) for gen in disc_vars_d]) / nb_var - torch.mean(true_vars_disc)
            #gen_loss = -sum([torch.mean(torch.exp(gen - 1)) for gen in disc_vars_g])

            # 識別器Dのバックプロパゲーションとパラメータの更新
            if epoch < train_epochs:
                disc_loss.backward()
                d_optimizer.step()

            # 生成器のGの損失の計算の残り（マトリクスの複雑さとDAGのNO TEAR）
            struc_loss = lambda1 / batch_size*drawn_graph.sum()     # Mのloss
            func_loss = lambda2 / batch_size*drawn_neurons.sum()   # Aのloss

            regul_loss = struc_loss + func_loss

            if epoch <= train_epochs * dagstart:
                # epochが基準前のときは、DAGになるようにMへのNO TEARSの制限はかけない
                loss = gen_loss + regul_loss

            else:
                # epochが基準後のときは、DAGになるようにNO TEARSの制限をかける
                filters = graph_sampler.get_proba()  # マトリクスMの要素を取得（ただし、0,1ではなく、1の確率）
                dag_constraint = notears_constr(filters*filters)  # NO TERARの計算

                # 徐々に線形にDAGの正則を強くする
                loss = gen_loss + regul_loss + \
                    ((epoch - train_epochs * dagstart) *
                     dagpenalization_increase) * dag_constraint

            if epoch >= train_epochs:
                # testのepochの場合、結果を取得
                output.add_(filters.data)
                output_loss.add_(gen_loss.data)
            else:
                # trainのepochの場合、生成器Gのバックプロパゲーションと更新
                # retain_graph=Trueにすることで、以降3つのstep()が実行できる
                loss.backward(retain_graph=True)
                g_optimizer.step()
                graph_optimizer.step()
                neuron_optimizer.step()

            # 進捗の表示
            if epoch % 50 == 0:
                pbar.set_postfix(gen=gen_loss.item()/cols,
                                 disc=disc_loss.item(),
                                 regul_loss=regul_loss.item(),
                                 tot=loss.item())

    return output.cpu().numpy()/test_epochs, output_loss.cpu().numpy()/test_epochs/cols  # Mと損失を出力


### GPUの使用可能を確認

画面上部のメニュー ランタイム > ランタイムのタイプを変更 で、 ノートブックの設定 を開く

ハードウェアアクセラレータに GPU を選択し、 保存 する

In [ ]:
# GPUの使用確認：True or False
torch.cuda.is_available()


True

### SAMの学習を実施

In [ ]:
# numpyの出力を小数点2桁に
np.set_printoptions(precision=2, floatmode='fixed', suppress=True)

# 因果探索の結果を格納するリスト
m_list = []
loss_list = []

for i in range(5):
    m, loss = run_SAM(in_data=df, lr_gen=0.01*0.5,
                      lr_disc=0.01*0.5*2,
                      #lambda1=0.01, lambda2=1e-05,
                      lambda1=5.0*20, lambda2=0.005*20,
                      hlayers=2,
                      nh=200, dnh=200,
                      train_epochs=10000,
                      test_epochs=1000,
                      device='cuda:0')

    print(loss)
    print(m)

    m_list.append(m)
    loss_list.append(loss)

# ネットワーク構造（5回の平均）
print(sum(m_list) / len(m_list))

# mはこうなって欲しい
#    x Z Y Y2 Y3 Y4
# x  0 1 1 0 0 0
# Z  0 0 1 0 0 0
# Y  0 0 0 0 1 0
# Y2 0 0 0 0 1 0
# Y3 0 0 0 0 0 1
# Y4 0 0 0 0 0 0


  0%|          | 4/11000 [00:00<05:40, 32.25it/s, disc=1.43, gen=0.683, regul_loss=1.53, tot=5.63]

[[7.38]]
[[0.00 0.10 0.00 0.58 0.92 0.82]
 [0.95 0.00 0.03 0.00 0.10 0.77]
 [0.57 1.00 0.00 1.00 1.00 0.18]
 [0.00 0.01 0.00 0.00 0.05 0.05]
 [0.01 0.01 0.01 0.99 0.00 1.00]
 [0.01 0.00 0.01 0.35 0.14 0.00]]


  0%|          | 4/11000 [00:00<05:24, 33.90it/s, disc=1.45, gen=0.886, regul_loss=1.53, tot=6.85]

[[6.07]]
[[0.00 0.99 0.95 0.19 0.00 0.00]
 [0.06 0.00 0.99 0.00 0.00 0.03]
 [0.00 0.03 0.00 1.00 0.99 0.03]
 [0.01 0.00 0.00 0.00 0.98 0.03]
 [0.00 0.00 0.00 0.01 0.00 0.07]
 [0.97 0.00 0.32 0.97 0.99 0.00]]


  0%|          | 4/11000 [00:00<05:32, 33.10it/s, disc=1.44, gen=0.646, regul_loss=1.48, tot=5.36]

[[7.81]]
[[0.00 0.00 0.99 0.98 0.01 0.00]
 [0.96 0.00 0.99 1.00 0.86 0.44]
 [0.00 0.00 0.00 0.02 0.99 0.08]
 [0.02 0.00 0.00 0.00 0.09 0.03]
 [0.07 0.00 0.01 0.97 0.00 0.99]
 [0.19 0.02 0.00 0.36 0.13 0.00]]


  0%|          | 4/11000 [00:00<05:47, 31.68it/s, disc=1.43, gen=0.593, regul_loss=1.53, tot=5.09]

[[6.65]]
[[0.00 0.99 1.00 0.98 0.72 0.03]
 [0.07 0.00 0.96 0.99 0.01 0.00]
 [0.00 0.07 0.00 0.05 0.02 0.02]
 [0.00 0.00 0.44 0.00 0.02 0.01]
 [0.00 0.19 0.41 1.00 0.00 0.98]
 [0.00 0.48 0.46 0.86 0.08 0.00]]


100%|██████████| 11000/11000 [05:47<00:00, 31.67it/s, disc=0.699, gen=4.86, regul_loss=0.317, tot=30.5]

[[3.77]]
[[0.00 0.97 0.90 0.00 0.00 0.00]
 [0.01 0.00 0.03 0.00 0.00 0.02]
 [0.01 0.88 0.00 0.98 1.00 0.45]
 [0.00 0.04 0.00 0.00 0.04 0.10]
 [0.00 0.00 0.02 0.98 0.00 0.98]
 [0.00 0.00 0.02 0.00 0.08 0.00]]
[[0.00 0.61 0.77 0.55 0.33 0.17]
 [0.41 0.00 0.60 0.40 0.19 0.25]
 [0.12 0.39 0.00 0.61 0.80 0.15]
 [0.01 0.01 0.09 0.00 0.24 0.04]
 [0.02 0.04 0.09 0.79 0.00 0.80]
 [0.24 0.10 0.16 0.51 0.28 0.00]]


以上